In [1]:
!pip3 install antigranular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.9 MB/s eta 0:00:00


In [4]:
#Setting up the session
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Harvard OpenDP Hackathon")

Dataset "Flight Company Dataset" loaded to the kernel as flight_company_dataset

Dataset "Health Organisation Dataset" loaded to the kernel as health_organisation_dataset

Connected to Antigranular server session id: f3fc53d9-ead4-4119-a2b4-f2c11266d6ea, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
%%ag
from op_pandas import PrivateSeries , PrivateDataFrame
import op_recordlinkage as rl
import re
from ag_utils import ag_print

In [ ]:
#setting up the dataframes and basic probing
%%ag
flight = flight_company_dataset
health = health_organisation_dataset

# ***Preprocessing and cleaning the data***

In [ ]:
%%ag
#Basic MetaData Analysis
ag_print(f'''dtypes of flight dataset:
{flight.dtypes}

dtypes of health dataset:
{health.dtypes}''')

dtypes of flight dataset:
flight_number              object
flight_date                object
flight_from                object
flight_to                  object
passenger_firstname        object
passenger_lastname         object
passenger_date_of_birth    object
passenger_phone_number     object
passenger_email_address    object
dtype: object

dtypes of health dataset:
patient_firstname        object
patient_lastname         object
patient_date_of_birth    object
patient_phone_number     object
patient_email_address    object
covidtest_date           object
covidtest_result         object
patient_address          object
dtype: object



In [ ]:
%%ag
#Removing all negative covid test results
health['covidtest_result'] = health['covidtest_result'].where(health['covidtest_result'] == 'positive')
health = health.dropna()

# Cleaning numbers
We use a function which takes in numbers in random format and gives them back in a uniform format

-We account for 3 different formats in numbers, if we find none of them, the original string is returned without change

-We are only considering the 10 digit number (no country codes)

In [ ]:
%%ag
def clean_numbers(num : str) -> str:

  #Accounting for no divider numbers
  if re.search(r'\d{10}', num[::-1]) != None:
    num_parts_reversed = re.search(r'\d{10}', num[::-1])

    return f'{num_parts_reversed.group()[::-1]}'

  #Accounting for numbers with dividers (with max of 2 dividers)
  num_parts_reversed = re.search(r'(\d+)\D(\d+)\D(\d+)', num[::-1])
  if num_parts_reversed != None:


    num_back = num_parts_reversed.group(1)
    num_mid = num_parts_reversed.group(2)

    if len(num_back) + len(num_mid) >= 10:
      num_mid_length = 10 - len(num_back)
      num_mid = num_mid[0:num_mid_length]

      return f"{num_mid[::-1]}{num_back[::-1]}"

    #the case where the 10 numbers are present before the 2nd divider
    else:

      num_front_len = 10 - len(num_back) - len(num_mid)
      num_front = num_parts_reversed.group(3)[0:num_front_len]
      return f"{num_front[::-1]}{num_mid[::-1]}{num_back[::-1]}"

  #Accounting numbers with 1 divider
  num_parts_reversed = re.search(r'(\d+)\D(\d+)', num[::-1])
  if num_parts_reversed != None:


    num_back = num_parts_reversed.group(1)
    num_mid = num_parts_reversed.group(2)
    if len(num_back) >= 10:
      num_length = 10 - len(num_back)
      num_back = num_back[0:num_length]
      return f"{num_back[::-1]}"

    #the case where the 10 numbers are present before the 1st divider
    else:
      num_mid_length = 10 - len(num_back)
      num_mid = num_mid[0:num_mid_length]

      return f"{num_mid[::-1]}{num_back[::-1]}"
  else:
    return num


In [ ]:
%%ag
flight['passenger_phone_number'] = flight['passenger_phone_number'].map(clean_numbers)
health["patient_phone_number"] = health["patient_phone_number"].map(clean_numbers)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



# Cleaning Dates
the cleaning functions output dates in a unifined format of "(day) (month_name) (year)".

(day) : a 2 length string, ex: 04, 23

(month_name) : the function tries to get as many enteries in their [full name month] format, but there still would be some entries that aren't caught by the function, they are returned as they were.

(year) : in 4 length string format, ex: 1978, 2020 etc.

the clean_bdates also detects children's bdays which were born after 2000.
Assuming that nobody born before 1921 is in our database.



In [ ]:
%%ag
def clean_bdates(date : str) -> str:
  month_name_dict = {("jan", "jn", "janury"): "January", ("feb", "fbury", "fb", "feberuary"):"February", ("mar", "mrch"):"March",
                     ("ap", "apr", "aprl"):"April", ("my", "ma"):"May", ("jun", "jn", "jne"):"June",
                     ("jul", "jly"):"July", ("aug", "agst", "augst"):"August", ("sept", "sep", "spt", "sptber", "septber"):"September",
                     ("oct", "otber", "ocber"):"October", ("nov", "novem", "nvmber"):"November", ("dec", "decem", "dcber"):"December"}

  #getting the day month and year info
  date_parts = re.search(r"([a-zA-Z0-9]+).([a-zA-Z0-9]+).([a-zA-Z0-9]+)", date)

  dd = "0"
  yyyy = "0"
  children_dates = [] #to isolate integer dates between 1-20
  mm = "01"
  for i in range(1, 4):
    temp = date_parts.group(i)
    if temp.isdigit():
      num = int(temp)
      if num > 31:
        yyyy = temp
      elif 20 < num < 32:
        dd = temp
      else:
        children_dates.append(temp)
    else:
      mm = temp

  #assigning the values in children_dates list(assuming nobody born before 1921 was travelling)
  if len(children_dates) == 1 and yyyy == "0":
      yyyy = str(2000+ int(children_dates[0]))
  elif len(children_dates) == 1 and dd == "0":
    dd = children_dates[0]
  elif len(children_dates) == 2: #if both the year and day are in 1-20, we can't differentiate between them
    yyyy = str(2000+ int(children_dates[0]))
    dd = children_dates[1]

  #getting the terms in the right format
  if len(yyyy) < 3:
      yyyy = "19" + yyyy
  if len(dd) < 2:
    dd = "0" + dd

  #checking for month formats
  mm = mm.lower()
  for ele in month_name_dict.keys():
    if mm in ele:
      mm  = month_name_dict[ele]
  mm = mm.capitalize()

  return f"{dd} {mm} {yyyy}"

In [ ]:
%%ag

def clean_dates(date : str) -> (str):
  month_name_dict = {("jan", "jn", "janury"): "January", ("feb", "fbury", "fb", "feberuary"):"February", ("mar", "mrch"):"March",
                     ("ap", "apr", "aprl"):"April", ("my", "ma"):"May", ("jun", "jn", "jne"):"June",
                     ("jul", "jly"):"July", ("aug", "agst", "augst"):"August", ("sept", "sep", "spt", "sptber", "septber"):"September",
                     ("oct", "otber", "ocber"):"October", ("nov", "novem", "nvmber"):"November", ("dec", "decem", "dcber"):"December"}

  #getting the date month and year info
  if ' ' in date:
    date_parts = date.split(' ')
  elif '-' in date:
    date_parts = date.split('-')
  elif '.' in date:
    date_parts = date.split('.')

  dd = '0'
  mm = '0'
  for i in range(1, 4):
    temp = date_parts[i-1]
    digit_check = temp.isdigit()
    if digit_check:
      num = int(temp)
      #check if 2020 is given as 20
      if num != 20:
        if num < 1000:
          dd = temp
    else:
      mm = temp

  if dd == '0':
    dd = "20"

  #checking for month format
  mm = mm.lower()
  for ele in month_name_dict.keys():
    if mm in ele:
      mm  = month_name_dict[ele]
  mm = mm.capitalize()


  if len(dd) < 2:
    dd = "0" + dd

  return f"{dd} {mm} 2020"

In [ ]:
%%ag
flight['passenger_date_of_birth'] = flight['passenger_date_of_birth'].map(clean_bdates)
health["patient_date_of_birth"] = health["patient_date_of_birth"].map(clean_bdates)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



In [ ]:
%%ag
flight['flight_date'] = flight['flight_date'].map(clean_dates)
health["covidtest_date"] = health["covidtest_date"].map(clean_dates)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



Note: We can't leave unfiltered date values because the comparison function defined for these values (flight date/covid test date) later can only take a single date format

# Cleaning names

phonetic function has only been applied on the lastnames and not the the firstnames as they can include prefix and middle names (which can not always be seperated easily)

In [ ]:
%%ag
from op_recordlinkage.preprocessing import clean
from op_recordlinkage.preprocessing import phonetic

In [ ]:
%%ag
def clean_lastnames(last_names : PrivateSeries) -> PrivateSeries:
  last_names =  clean(last_names, lowercase=True , replace_by_none = ' ', strip_accents = 'unicode')
  last_names = phonetic(last_names, method= 'soundex')
  return last_names

def clean_firstnames(first_names : PrivateSeries) -> PrivateSeries:
  first_names = clean(first_names, lowercase = True,  strip_accents = 'unicode', replace_by_whitespace = r"(mr|ms|mrs|mx|dr)")
  return first_names

In [ ]:
%%ag
flight['passenger_lastname'] = clean_lastnames(flight['passenger_lastname'])
health['patient_lastname'] = clean_lastnames(health['patient_lastname'])

In [ ]:
%%ag
flight['passenger_firstname'] = clean_firstnames(flight['passenger_firstname'])
health['patient_firstname'] = clean_firstnames(health['patient_firstname'])

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



# Cleaning Emails

we will be only using the part of the emails of the users which is before the @, so we are not extracting the emailing service

In [ ]:
%%ag
def clean_emails(mail : str) -> str:
  before_at = re.search('(.+)@', mail)
  temp = before_at.group(1)
  return temp

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



In [ ]:
%%ag
flight['passenger_email_address'] = flight["passenger_email_address"].map(clean_emails)
health["patient_email_address"] = health["patient_email_address"].map(clean_emails)

# ***Linking the databases***
for this we will be using the op_recordlinkage that we imported earlier

# Indexing

using date of birth, lastnames, phone numbers and email address as they are most likely to be the same across the databases (lastnames were taken out later as they generated too many pairs (so much that the compare function had difficulty going through them all))

In [ ]:
%%ag
indexer = rl.Index()
#indexer.block('passenger_lastname', 'patient_lastname')
indexer.block('passenger_date_of_birth','patient_date_of_birth')
indexer.block('passenger_phone_number','patient_phone_number')
indexer.block("passenger_email_address", "patient_email_address")
links = indexer.index(flight,health)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



# Comparison
- Firstname: string Jarowinkler
- Lastname:  string Jarowinkler
- Phonenumbers: string damerau_levenshtein
- Birtdates: damerau_levenshtein
- Emails: string damerau_levenshtein
- Flight/Covid_test dates: custom date function
- Flight-to/patient-address: custom location comparison

In [ ]:
%%ag
import datetime
def cmp_risk_period(date_str1 : str, date_str2 : str) -> float:
    # Convert date strings to datetime objects
    mm1 = date_str1[3:6]
    mm2 = date_str2[3:6]
    date_str1 = date_str1[0:2]
    date_str2 = date_str2[0:2]
    date1 = datetime.datetime.strptime(date_str1, "%d")
    date2 = datetime.datetime.strptime(date_str2, "%d")

    # Calculate the absolute difference in days
    days_apart = (date2 - date1).days
    # Check if the dates are within two weeks (14 days) apart
    if (days_apart <= 14) and (mm1.lower() == mm2.lower()):
        return 2.4
    else:
        return 0


In [ ]:
%%ag
def cmp_place(flight_to : str, patient_address : str) -> float:
  if flight_to.lower() in patient_address:
    return 0.65
  return 0.0

In [ ]:
%%ag
compare = rl.Compare()
compare.string(left_on='passenger_firstname',right_on='patient_firstname',label='firstname',method='jarowinkler')
compare.string(left_on='passenger_lastname',right_on='patient_lastname',label='lastname',method='jarowinkler')
compare.string(left_on='passenger_phone_number',right_on='patient_phone_number',label='phone_number',method='damerau_levenshtein')
compare.string(left_on='passenger_email_address', right_on='patient_email_address', label='emails', method= 'damerau_levenshtein')
compare.string(left_on='passenger_date_of_birth', right_on='patient_date_of_birth', label='birthdates', method= "damerau_levenshtein")
compare.custom(cmp_place, 'flight_to',"patient_address", label="Location")
compare.custom(cmp_risk_period, left_on='flight_date', right_on='covidtest_date', label='Time-period')



In [ ]:
%%ag
features = compare.compute(links,flight,health)

In [ ]:
%%ag
def modify_weights5(weight : float) -> float :
  return weight*1.9

def modify_weights3(weight: float) -> float:
  return weight*1.7

In [ ]:
%%ag
features['phone_number'] = features['phone_number'].map(modify_weights5, output_bounds= (0, 1.9))
features['emails'] = features['emails'].map(modify_weights5, output_bounds= (0, 1.9))
features['birthdates'] = features['birthdates'].map(modify_weights3, output_bounds= (0, 1.7))

# Linking
comparing the sum of the weights of the feature matrix while monitering the epsilon usage

In [ ]:
#session.privacy_odometer()

In [ ]:
%%ag
ag_print(f"Average weight : {features.sum(axis=1).mean(eps=0.1)}")

Average weight : 6.590007391422214



In [ ]:
#session.privacy_odometer()

In [ ]:
%%ag
linked_df = compare.get_match(7.2)

In [ ]:
%%ag
res = linked_df[["l_flight_number"]]
x = submit_predictions(res)

score: {'leaderboard': 0.6733297454265867, 'logs': {'LIN_EPS': -0.0034999999999999996, 'MCC': 0.6768297454265867}}



In [5]:

session.terminate_session()

{'status': 'ok'}